In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.utils import to_categorical
import cv2
from collections import defaultdict
import matplotlib.pyplot as plt
import random
from PIL import Image
from tensorflow.keras import regularizers




In [2]:
# Wczytanie danych
train_data = pd.read_csv("train_data_with_features.csv")
test_data = pd.read_csv("test_data_with_features.csv")

In [3]:
# Przygotowanie cech i etykiet
y_train_val = train_data['class']
X_train_val = train_data.drop(['class', "path", "filename", "R", "G", "B", "brightness"], axis=1)
y_test = test_data['class']
X_test = test_data.drop(['class', "path", "filename", "R", "G", "B", "brightness"], axis=1)


scaler = StandardScaler()
X_train_val = scaler.fit_transform(X_train_val)
X_test = scaler.transform(X_test)

In [4]:
# Konwersja etykiet na wartości liczbowe
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_train_val)



In [5]:
# Definicja 5-krotnej walidacji krzyżowej
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

accuracies = []
reports = []

In [6]:
# Iteracja po podziałach walidacji krzyżowej
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_val, y_train_val)):
    print(f"Fold {fold + 1}")
    X_train, X_val = X_train_val[train_idx], X_train_val[val_idx]
    y_train, y_val = y_train_val[train_idx], y_train_val[val_idx]
    
    # Tworzenie i trenowanie modelu
    clf = RandomForestClassifier(n_estimators=100, random_state=42,max_depth=7)
    clf.fit(X_train, y_train)
    
    # Predykcja
    y_pred = clf.predict(X_val)
    
    # Ewaluacja
    accuracy = accuracy_score(y_val, y_pred)
    accuracies.append(accuracy)
    report = classification_report(y_val, y_pred, target_names=label_encoder.classes_)
    reports.append(report)
    
    print(f"Dokładność (walidacja): {accuracy:.4f}")
    print("Raport klasyfikacji:\n", report)
    print("-" * 50)



Fold 1
Dokładność (walidacja): 0.8952
Raport klasyfikacji:
                       precision    recall  f1-score   support

      Bombali Ghevda       0.78      0.69      0.73       136
        Butka Ghevda       0.97      0.98      0.97       136
       Gabara Ghevda       0.98      0.98      0.98       136
             God Wal       0.68      0.77      0.72       136
          Hirva Lamb       0.92      0.90      0.91       136
            Kadu wal       0.79      0.98      0.88       136
            Kala Wal       0.94      0.92      0.93       136
     Lal lamb shenga       0.86      0.88      0.87       136
Lamb Shiracha Ghevda       0.93      0.93      0.93       136
      Shravan Ghevda       1.00      0.93      0.96       135
          Tambda wal       1.00      1.00      1.00       136
       Vatana Ghevda       0.96      0.79      0.86       136

            accuracy                           0.90      1631
           macro avg       0.90      0.90      0.90      1631
        

In [7]:
# Ostateczna ewaluacja na zbiorze testowym
final_model = RandomForestClassifier(n_estimators=100, random_state=42)
final_model.fit(X_train_val, y_train_val)
y_test_pred = final_model.predict(X_test)



In [8]:
test_accuracy = accuracy_score(y_test, y_test_pred)
test_report = classification_report(y_test, y_test_pred, target_names=label_encoder.classes_)



In [9]:
print(f"Średnia dokładność na walidacji: {np.mean(accuracies):.4f}")
print(f"Dokładność na zbiorze testowym: {test_accuracy:.4f}")
print("Raport klasyfikacji na zbiorze testowym:\n", test_report)

Średnia dokładność na walidacji: 0.9018
Dokładność na zbiorze testowym: 0.9653
Raport klasyfikacji na zbiorze testowym:
                       precision    recall  f1-score   support

      Bombali Ghevda       1.00      1.00      1.00        12
        Butka Ghevda       1.00      1.00      1.00        12
       Gabara Ghevda       1.00      1.00      1.00        12
             God Wal       1.00      0.83      0.91        12
          Hirva Lamb       1.00      0.92      0.96        12
            Kadu wal       0.80      1.00      0.89        12
            Kala Wal       0.92      1.00      0.96        12
     Lal lamb shenga       1.00      0.92      0.96        12
Lamb Shiracha Ghevda       0.92      1.00      0.96        12
      Shravan Ghevda       1.00      1.00      1.00        12
          Tambda wal       1.00      1.00      1.00        12
       Vatana Ghevda       1.00      0.92      0.96        12

            accuracy                           0.97       144
         

In [10]:
y_one_hot = to_categorical(y_encoded)


In [11]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train_val)

In [12]:
mlp_accuracies = []

for train_idx, val_idx in skf.split(X_scaled, y_encoded):
    X_train, X_val = X_scaled[train_idx], X_scaled[val_idx]
    y_train, y_val = y_one_hot[train_idx], y_one_hot[val_idx]
    
    mlp_model = models.Sequential([
        layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        layers.Dense(64, activation='relu'),
        layers.Dense(len(np.unique(y_train_val)), activation='softmax')
    ])
    
    mlp_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    mlp_model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=32, verbose=0)
    
    val_accuracy = mlp_model.evaluate(X_val, y_val, verbose=0)[1]
    mlp_accuracies.append(val_accuracy)
    


C:\Users\User\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
print(f"Średnia dokładność MLP: {np.mean(mlp_accuracies):.4f}")

Średnia dokładność MLP: 0.9180


In [14]:
data_path = "Dataset_split/train"
data_images = []
labels = []
image_size=(100, 70)


for class_folder in os.listdir(data_path):  
    class_path = os.path.join(data_path, class_folder)
    if os.path.isdir(class_path):  
        for file in os.listdir(class_path):  
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):  
                img = cv2.imread(os.path.join(class_path, file))  # Wczytanie obrazu w kolorze
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Konwersja z BGR do RGB
                img = cv2.resize(img, image_size)
                data_images.append(img)
                labels.append(class_folder)




In [15]:
X_images = np.array(data_images) / 255.0  # Normalizacja

label_encoder_images = LabelEncoder()
y_images = label_encoder_images.fit_transform(labels)
y_images_one_hot = to_categorical(y_images)

In [16]:
# 5-krotna walidacja krzyżowa dla CNN
cnn_accuracies = []
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
X_images = np.array(X_images)



In [17]:
i=1
for train_idx, val_idx in skf.split(y_images, y_images):  # Zmiana indeksacji na y_images
    X_train, X_val = X_images[train_idx], X_images[val_idx]
    y_train, y_val = y_images_one_hot[train_idx], y_images_one_hot[val_idx]
    
    
    cnn_model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(70, 100, 3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(np.unique(y_images)), activation='softmax')
])
    cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    cnn_model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=32, verbose=0)
    
    val_accuracy = cnn_model.evaluate(X_val, y_val, verbose=0)[1]
    cnn_accuracies.append(val_accuracy)
    
    print(f"Dokładność w iteracji {i}: {val_accuracy:.4f}")
    i+=1


C:\Users\User\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Dokładność w iteracji 1: 0.8829
Dokładność w iteracji 2: 0.8412
Dokładność w iteracji 3: 0.9215
Dokładność w iteracji 4: 0.8921
Dokładność w iteracji 5: 0.8829


In [18]:
print(f"Średnia dokładność CNN: {np.mean(cnn_accuracies):.4f}")

Średnia dokładność CNN: 0.8841


In [ ]:
## sprawdzenie na zbiorze testowym

data_path = "Dataset_split/test"
data_images = []
labels = []
image_size=(100, 70)


for class_folder in os.listdir(data_path):  
    class_path = os.path.join(data_path, class_folder)
    if os.path.isdir(class_path):  
        for file in os.listdir(class_path):  
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):  
                img = cv2.imread(os.path.join(class_path, file))  # Wczytanie obrazu w kolorze
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Konwersja z BGR do RGB
                img = cv2.resize(img, image_size)
                data_images.append(img)
                labels.append(class_folder)
X_images = np.array(data_images) / 255.0  # Normalizacja

label_encoder_images = LabelEncoder()
y_images = label_encoder_images.fit_transform(labels)
y_images_one_hot = to_categorical(y_images)


cnn_model.evaluate(X_images, y_images_one_hot, verbose=0)[1]